# Parameter Sweep Tutorial for QRC

This notebook performs a parameter sweep for evaluating Quantum Reservoir Computing performance.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add project source directory to path
sys.path.append(os.path.join('..'))

from parameter_sweep import ParameterSweep
from config_manager import ConfigManager
from visualization_sweep import create_summary_dashboard, plot_parameter_effect, plot_heatmap
from sweep_analysis import identify_significant_parameters, generate_best_configuration_table

## 1. Basic Parameter Sweep Methodology

When conducting a parameter sweep for QRC, consider the following methodology:

1. **Define baseline parameters**: Start with a working configuration
2. **Identify key parameters**: Select parameters that are most likely to affect performance
3. **Define parameter ranges**: Set reasonable ranges for each parameter
4. **Run sweep**: Perform the parameter sweep, keeping most parameters fixed while varying a few
5. **Analyze results**: Identify optimal parameter values and their interactions
6. **Refine search**: Narrow the parameter ranges around optimal values for more precision

Let's implement this approach for our QRC experiments.

## 2. Load Base Configuration

First, let's load a base configuration to use for our parameter sweeps.

In [2]:
# Load default configuration
base_config = ConfigManager.get_default_config()

# Display configuration
for key, value in base_config.items():
    print(f"{key}: {value}")

dataset_type: cvc_clinic_db_patches
data_dir: None
target_size: [128, 128]
split_ratio: 0.8
reduction_method: autoencoder
dim_reduction: 12
num_examples: 10000
num_test_examples: 400
guided_alpha: 0.3
guided_beta: 0.7
quantum_update_frequency: 1
guided_batch_size: 32
autoencoder_epochs: 50
autoencoder_batch_size: 64
autoencoder_learning_rate: 0.001
autoencoder_hidden_dims: None
gpu: False
geometry: chain
lattice_spacing: 10.0
rabi_freq: 6.283185307179586
evolution_time: 4.0
time_steps: 16
readout_type: all
n_shots: 1000
detuning_max: 6.0
encoding_scale: 9.0
regularization: 0.0005
nepochs: 100
batchsize: 1000
learning_rate: 0.01
seed: 42
no_progress: False
no_plot: False


## 3. Define Parameter Grid for Sweep

Let's define a parameter grid for our sweep. We'll focus on a few key parameters:

In [3]:
# Define parameter grid for a simple sweep
param_grid = {
    "reduction_type": ["pca", "autoencoder", "guided_autoencoder"],  # Type of dimensionality reduction
    "dim_reduction": [6, 8, 10, 12],  # Encoding dimension
    "readout_type": ["Z", "ZZ", "all"],  # Quantum readout type
    "n_shots": [500, 1000],  # Number of quantum shots
    "seed": [42, 43]  # Multiple seeds for statistical significance
}

# Calculate total number of experiments
n_experiments = 1
for values in param_grid.values():
    n_experiments *= len(values)

print(f"Total number of experiments: {n_experiments}")

Total number of experiments: 144


## 4. Initialize Parameter Sweep

Now let's initialize the parameter sweep with our base configuration.

In [4]:
# Set output directory for results
output_dir = os.path.join('results', 'sweep')
os.makedirs(output_dir, exist_ok=True)

# Initialize parameter sweep
sweep = ParameterSweep(base_config, output_dir)

## 5. Run Parameter Sweep

Let's run the parameter sweep. This will execute the experiments defined by our parameter grid.

In [ ]:
# Run the parameter sweep
# Note: This will run many experiments and may take a long time
# Uncomment to run

results_df = sweep.run_sweep(param_grid, "notebook_demo")

# Alternatively, for demonstration purposes, let's load a pre-computed result
# This assumes you've already run a sweep and saved the results
# results_df = pd.read_csv(os.path.join(output_dir, 'sample_results.csv'))

# Print the first few rows of the results
results_df.head()

## 6. Analyze Results

After running the parameter sweep, we can analyze the results to identify optimal parameter values.

In [ ]:
# This assumes you've run the sweep or loaded results in the previous cell
# If not, load a sample results file
sample_results_path = os.path.join('..', 'results', 'sample_results.csv')

# Check if file exists, if not create a dummy dataframe for demonstration
if os.path.exists(sample_results_path):
    results_df = pd.read_csv(sample_results_path)
else:
    # Create dummy data for demonstration
    print("No results file found. Creating dummy data for demonstration.")
    # Generate dummy data
    np.random.seed(42)
    n_samples = 24
    results_df = pd.DataFrame({
        'dim_reduction': np.random.choice([6, 8, 10, 12], n_samples),
        'readout_type': np.random.choice(['Z', 'ZZ', 'all'], n_samples),
        'n_shots': np.random.choice([500, 1000], n_samples),
        'seed': np.random.choice([42, 43], n_samples),
        'QRC_final_test_acc': np.random.uniform(0.7, 0.95, n_samples),
        'PCA+linear_final_test_acc': np.random.uniform(0.6, 0.85, n_samples),
        'PCA+NN_final_test_acc': np.random.uniform(0.65, 0.9, n_samples)
    })
    
# Display the first few rows
results_df.head()

In [ ]:
# Identify significant parameters
target_metric = 'QRC_final_test_acc'
significant_params = identify_significant_parameters(results_df, target_metric)
print(f"Significant parameters affecting {target_metric}: {significant_params}")

In [ ]:
# Generate best configuration table
best_configs = generate_best_configuration_table(results_df)
best_configs

In [ ]:
# Plot effect of each parameter on performance
for param in significant_params:
    if param in results_df.columns and results_df[param].nunique() > 1:
        plot_parameter_effect(results_df, param, target_metric)

In [ ]:
# Plot parameter interactions if we have at least 2 significant parameters
if len(significant_params) >= 2:
    plot_heatmap(results_df, significant_params[0], significant_params[1], target_metric)

## 7. Create Dashboard

We can create a comprehensive dashboard of our parameter sweep results.

In [ ]:
# Create dashboard directory
dashboard_dir = os.path.join(output_dir, 'dashboard')
os.makedirs(dashboard_dir, exist_ok=True)

# Create dashboard
create_summary_dashboard(results_df, dashboard_dir, target_metric)

print(f"Dashboard created in {dashboard_dir}")

## 8. Refined Parameter Sweep

Based on the initial results, we can refine our parameter sweep to focus on promising parameter ranges.

In [ ]:
# Identify optimal parameter values from initial sweep
best_idx = results_df[target_metric].idxmax()
best_config = results_df.loc[best_idx]

print("Best configuration from initial sweep:")
for param in significant_params:
    if param in best_config:
        print(f"  {param}: {best_config[param]}")
print(f"  Performance: {best_config[target_metric]:.4f}")

In [ ]:
# Define refined parameter grid around best values
# This is just an example, you would adjust based on your actual results
refined_param_grid = {}

# For demonstration, let's assume dim_reduction=10 was best
refined_param_grid["dim_reduction"] = [8, 9, 10, 11, 12]

# For demonstration, let's assume readout_type="all" was best
refined_param_grid["readout_type"] = ["ZZ", "all"]

# For demonstration, let's assume n_shots=1000 was best
refined_param_grid["n_shots"] = [800, 1000, 1200]

# Add more parameters to explore
refined_param_grid["rabi_freq"] = [2*np.pi, 4*np.pi]
refined_param_grid["seed"] = [42, 43, 44]

# Calculate total number of experiments in refined sweep
n_refined = 1
for values in refined_param_grid.values():
    n_refined *= len(values)

print(f"Total number of experiments in refined sweep: {n_refined}")

## 9. Run Refined Sweep

Now we can run the refined parameter sweep.

In [ ]:
# Run the refined parameter sweep
# Note: This will run many experiments and may take a long time
# Uncomment to run

# refined_results_df = sweep.run_sweep(refined_param_grid, "notebook_refined")

# For demonstration purposes, let's just print the parameter grid
print("Refined parameter grid:")
for param, values in refined_param_grid.items():
    print(f"  {param}: {values}")

## 10. Summary and Best Practices

Here are some best practices for parameter sweeps in QRC:

1. **Start broad, then refine**: Begin with a wide range of parameter values, then narrow down
2. **Use multiple seeds**: Run each configuration with multiple random seeds for statistical significance
3. **Group related parameters**: Sweep related parameters together (e.g., quantum parameters)
4. **Balance exploration vs. computation**: Consider computational resources when defining parameter grids
5. **Save all results**: Keep detailed records of all experiments for later analysis
6. **Analyze parameter interactions**: Look for interactions between parameters using heatmaps
7. **Create publication-quality visualizations**: Generate clear, informative figures for your paper

By following this methodology, you'll be able to identify optimal parameter configurations for your QRC experiments and present compelling results in your scientific paper.